# Hough Transformation in CCD Images
This notebook demonstrates how to apply the Hough transformation on an image to detect lines.

In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt


class CcdPixelHit:
    """Represents a pixel hit in a CCD."""

    def __init__(self, tx, ty):
        self.xpixel = tx
        self.ypixel = ty
        self.edep = 0.0

    @property
    def x_pixel(self):
        return self.xpixel

    @property
    def y_pixel(self):
        return self.ypixel

    @property
    def energy_deposit(self):
        return self.edep

    @energy_deposit.setter
    def energy_deposit(self, value):
        self.edep = value

    def is_same(self, other):
        return self.xpixel == other.xpixel and self.ypixel == other.ypixel


class CcdHoughCell:
    """Represents a Hough cell for detecting lines in a CCD."""

    def __init__(self, pix1, pix2):
        self._pixel1 = pix1
        self._pixel2 = pix2
        self._delX = float(pix2.x_pixel) - float(pix1.x_pixel)
        self._delY = float(pix2.y_pixel) - float(pix1.y_pixel)
        self._slope = self._delY / (self._delX + 1e-20)
        self._intercept = pix2.y_pixel - (self._slope * pix1.x_pixel)
        self._theta = math.atan2(-self._delX, self._delY) * 180 / math.pi
        self._rho = abs(self._intercept) * abs(math.sin(self._theta * math.pi / 180))

    @property
    def pixel1(self):
        return self._pixel1

    @property
    def pixel2(self):
        return self._pixel2

    @property
    def delta_x(self):
        return self._delX

    @property
    def delta_y(self):
        return self._delY

    @property
    def slope(self):
        return self._slope

    @property
    def intercept(self):
        return self._intercept

    @property
    def theta(self):
        return self._theta

    @property
    def rho(self):
        return self._rho


class CcdEvent:
    """Represents an event in a CCD."""

    def __init__(self, hits):
        self.eventID = -1
        self.momin = 0
        self.thein = -1000
        self.phiin = -1000
        self.allHits = hits
        self.nhits = len(hits)
        self.meanRho = -10.0
        self.sigmaRho = -10.0
        self.meanTheta = -1000.0
        self.sigmaTheta = -1000.0
        self.houghSelHits = []

        if self.nhits > 1:
            self.houghSelHits, self.rhoList, self.thetaList = self.apply_hough_transform()

    @property
    def event_id(self):
        return self.eventID

    @property
    def theta_true(self):
        return self.thein

    @property
    def phi_true(self):
        return self.phiin

    @property
    def mean_rho(self):
        return self.meanRho

    @property
    def sigma_rho(self):
        return self.sigmaRho

    @property
    def mean_theta(self):
        return self.meanTheta

    @property
    def sigma_theta(self):
        return self.sigmaTheta

    def add_pixel_hit(self, hit):
        self.allHits.append(hit)

    @property
    def num_pixel_hits(self):
        return len(self.allHits)

    @property
    def num_hough_pixel_hits(self):
        return len(self.houghSelHits)

    def add_hough_sel_hit(self, hit):
        if not any(h.is_same(hit) for h in self.houghSelHits):
            self.houghSelHits.append(hit)

    def get_pixel_hit(self, index):
        return self.allHits[index]

    def get_hough_sel_hit(self, index):
        return self.houghSelHits[index]

    def apply_hough_transform(self):
        npix = self.num_pixel_hits
        houghList = []
        thetaList = []
        rhoList = []

        for i in range(npix - 1):
            for j in range(i + 1, npix):
                hough_cell = CcdHoughCell(self.get_pixel_hit(i), self.get_pixel_hit(j))
                houghList.append(hough_cell)
                thetaList.append(hough_cell.theta)
                rhoList.append(hough_cell.rho)

        if len(thetaList) > 5:
            self.meanRho = np.mean(rhoList)
            self.sigmaRho = math.sqrt(np.var(rhoList))
            self.meanTheta = np.mean(thetaList)
            self.sigmaTheta = math.sqrt(np.var(thetaList))
        else:
            self.meanRho = -10.0
            self.sigmaRho = -10.0
            self.meanTheta = -1000.0
            self.sigmaTheta = -1000.0

        for _ in range(100):
            new_hough_list = []
            listRho = []
            listTheta = []
            for cell in houghList:
                if (self.meanTheta - self.sigmaTheta < cell.theta < self.meanTheta + self.sigmaTheta and
                        self.meanRho - self.sigmaRho < cell.rho < self.meanRho + self.sigmaRho):
                    new_hough_list.append(cell)
                    listRho.append(cell.rho)
                    listTheta.append(cell.theta)

            if len(new_hough_list) < len(houghList):
                houghList = new_hough_list
                if len(listTheta) > 5:
                    self.meanRho = np.mean(listRho)
                    self.sigmaRho = math.sqrt(np.var(listRho))
                    self.meanTheta = np.mean(listTheta)
                    self.sigmaTheta = math.sqrt(np.var(listTheta))
                else:
                    self.meanRho = -10.0
                    self.sigmaRho = -10.0
                    self.meanTheta = -1000.0
                    self.sigmaTheta = -1000.0

                if abs(self.sigmaTheta) < 0.0001:
                    self.sigmaTheta = 0.0001
                if abs(self.sigmaRho) < 0.0001:
                    self.sigmaRho = 0.0001
            else:
                break

        return houghList, rhoList, thetaList


def main(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Validate if the image is loaded
    if image is None:
        raise ValueError(f"Failed to load image file '{image_path}'.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Get edge points
    edge_points = np.argwhere(edges)

    # Create CcdPixelHit instances for each edge point
    hits = [CcdPixelHit(x, y) for y, x in edge_points]

    # Create CcdEvent and apply Hough transformation
    event = CcdEvent(hits)

    # Extract the Hough transformation results
    hough_cells, rho_list, theta_list = event.apply_hough_transform()

    # Plot the results
    plt.scatter(theta_list, rho_list, s=1)
    plt.title('Hough ρ vs θ')
    plt.xlabel('Theta (degrees)')
    plt.ylabel('Rho (pixels)')
    plt.show()


if __name__ == "__main__":
    image_path = 'abc.png'  # Replace with the actual path to your image
    main(image_path)
